In [5]:
import pandas as pd
import numpy as np

discrete_dict = {"age":0, "workclass":1, "fnlwgt":0, "education":1, "education-num":0, "marital-status":1, "occupation":1, "relationship":1, "race":1, "sex":1, "capital-gain":0, "capital-loss":0, "hours-per-week":0, "native-country":1, "salary":0}

train_data = pd.read_csv("adult.data",names=["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","salary"],index_col=False)
cut = int(0.9 * len(train_data))
train_data, validation_data = train_data[:cut], train_data[cut:]
test_data = pd.read_csv("adult.test",names=["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","salary"],index_col=False,header=0)

attributes = ["workclass","education","marital-status","occupation","relationship","race","sex","native-country","salary"]
train_data = train_data[attributes] # only discrete values
test_data = test_data[attributes] # only discrete values

# fill in ?
def fill_data(data):
    for a in attributes:
        a_col = data[a]
        a_col[a_col == " ?"] = data[a].value_counts().argmax()

fill_data(train_data)
fill_data(test_data)

In [3]:
def entropy(p):
    if p.ndim == 1:
        new_p = p[p != 0]
        return -np.sum(new_p * np.log2(new_p))
    else:
        # new_p = p[(p[:,0] != 0) & (p[:,1] != 0)]
        return -np.sum(p * np.log2(p),axis=1)

def information_gain(D,a):
    pk = D["salary"].value_counts(normalize=True).values
    prop_Dv = D[a].value_counts(normalize=True).values # proportion
    prob_Dv = np.array([D.loc[D[a] == av]["salary"].value_counts(normalize=True).get(" >50K",0) for av in D[a].unique()])
    pp_stack = np.column_stack((prop_Dv,prob_Dv))
    pp_stack = pp_stack[(pp_stack[:,1] != 0) & (pp_stack[:,1] != 1)]
    prop_Dv = pp_stack[:,0]
    prob_Dv = pp_stack[:,1]
    prob_Dv_neg = 1 - prob_Dv
    return entropy(pk) - np.sum(prop_Dv * entropy(np.column_stack((prob_Dv,prob_Dv_neg))))

cnt = 0
class Node:

    def __init__(self):
        self.branch = {}

    def setLeaf(self,catagory):
        global cnt
        cnt += 1
        if cnt % 100 == 0:
            print("{} - Create leaf: {}".format(cnt,catagory),flush=True)
        self.label = "Leaf"
        self.catagory = catagory
        
    def setBranch(self,attr,value,node):
        # print("Create branch: {} ({})".format(attr,value),flush=True)
        self.label = "Branch"
        self.attr = attr
        self.branch[value] = node

class ID3:

    def __init__(self,train_set=None,validation_set=None,test_set=None):
        self.train_set = train_set
        self.validation_set = validation_set
        self.test_set = test_set

    def TreeGenerate(self,dataset,attributes):
        catagory = dataset["salary"].unique()
        node = Node()
        if len(catagory) == 1:
            node.setLeaf(catagory[0])
            return node
        if len(attributes) == 0 or np.sum([len(dataset[a].unique()) for a in attributes]) == len(attributes):
            node.setLeaf(dataset["salary"].value_counts().argmax())
            return node
        # print("Get here!",flush=True)
        max_gain = -0x3f3f3f3f
        for a in attributes:
            gain = information_gain(dataset,a)
            if gain > max_gain:
                a_best, max_gain = a, gain
        for av in self.train_set[a_best].unique(): # be careful, not dataset!
            Dv = dataset[dataset[a_best] == av]
            if len(Dv) == 0:
                node.setLeaf(dataset["salary"].value_counts().argmax())
                return node
            node.setBranch(a_best,av,self.TreeGenerate(Dv,attributes[attributes != a_best]))
        return node

    def train(self,train_set=None):
        if train_set != None:
            self.train_set = train_set
        self.root = self.TreeGenerate(self.train_set,self.train_set.columns.values[:-1])

    def validation(self,validation_set=None):
        if validation_set != None:
            self.validation_set = validation_set
        acc = 0
        for i,row in self.validation_set.iterrows():
            p = self.root
            while p.label != "Leaf":
                p = p.branch[row[p.attr]]
            if p.catagory == row["salary"][:-1]: # be careful of "."
                acc += 1
        acc /= len(self.validation_set)
        return acc

    def test(self,test_set=None):
        if test_set != None:
            self.test_set = test_set
        acc = 0
        for i,row in self.test_set.iterrows():
            p = self.root
            while p.label != "Leaf":
                p = p.branch[row[p.attr]]
            if p.catagory == row["salary"][:-1]: # be careful of "."
                acc += 1
        acc /= len(self.test_set)
        print("Accurary: {:.2f}%".format(acc * 100))
        return acc

In [4]:
dt = ID3(train_set=train_data,validation_set=validation_data,test_set=test_data)
dt.train()

100 - Create leaf:  >50K
200 - Create leaf:  <=50K
300 - Create leaf:  <=50K
400 - Create leaf:  <=50K
500 - Create leaf:  <=50K
600 - Create leaf:  >50K
700 - Create leaf:  <=50K
800 - Create leaf:  <=50K
900 - Create leaf:  <=50K
1000 - Create leaf:  >50K
1100 - Create leaf:  >50K
1200 - Create leaf:  >50K
1300 - Create leaf:  <=50K
1400 - Create leaf:  <=50K
1500 - Create leaf:  <=50K
1600 - Create leaf:  <=50K
1700 - Create leaf:  <=50K
1800 - Create leaf:  <=50K
1900 - Create leaf:  <=50K


In [18]:
# dt.test(test_data)
acc = 0
for i,row in test_data.iterrows():
    p = dt.root
    while p.label != "Leaf":
        p = p.branch[row[p.attr]]
    if p.catagory == row["salary"][:-1]:
        acc += 1
print("{:.2f}".format(acc / len(test_data)*100))

82.18


In [20]:
train_data[:5]

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,salary
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K
